# Initialization

In [23]:
import pyspark
sparkcontext = pyspark.SparkContext.getOrCreate(conf=(
        pyspark
        .SparkConf()
        .setAppName('Dibimbing')
        .setMaster('local')
    ))
sparkcontext.setLogLevel("WARN")

spark = pyspark.sql.SparkSession(sparkcontext.getOrCreate())

In [24]:
spark

# Data Load

In [25]:
!ls /resources/data/activity-data/ | head -10

_committed_730451297822678341
part-00000-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00001-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00002-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00003-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00004-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00005-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00006-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00007-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00008-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json


In [26]:
static = spark.read.json('/resources/data/activity-data/')
dataSchema = static.schema
print(dataSchema)

StructType([StructField('Arrival_Time', LongType(), True), StructField('Creation_Time', LongType(), True), StructField('Device', StringType(), True), StructField('Index', LongType(), True), StructField('Model', StringType(), True), StructField('User', StringType(), True), StructField('gt', StringType(), True), StructField('x', DoubleType(), True), StructField('y', DoubleType(), True), StructField('z', DoubleType(), True)])


In [27]:
static.show(5)

+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
| Arrival_Time|      Creation_Time|  Device|Index| Model|User|   gt|           x|           y|           z|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
|1424686735090|1424686733090638193|nexus4_1|   18|nexus4|   g|stand| 3.356934E-4|-5.645752E-4|-0.018814087|
|1424686735292|1424688581345918092|nexus4_2|   66|nexus4|   g|stand|-0.005722046| 0.029083252| 0.005569458|
|1424686735500|1424686733498505625|nexus4_1|   99|nexus4|   g|stand|   0.0078125|-0.017654419| 0.010025024|
|1424686735691|1424688581745026978|nexus4_2|  145|nexus4|   g|stand|-3.814697E-4|   0.0184021|-0.013656616|
|1424686735890|1424688581945252808|nexus4_2|  185|nexus4|   g|stand|-3.814697E-4|-0.031799316| -0.00831604|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
only showing top 5 rows



Metadata for the dataset

| Column | Description |
| --- | ----------- |
| Index         |  The row number.
| Arrival_Time  |  The time the measurement arrived to the sensing application
| Creation_Time |  The timestamp the OS attaches to the sample
| X,Y,Z | The values provided by the sensor for the three axes, X,y,z
| User          |  The user this sample originates from, the users are named a to i.
| Model         |  The phone/watch model this sample originates from
| Device        |  The specific device this sample is from. They are prefixed with the model name and then the number, e.g., nexus4_1 or nexus4_2.
| Gt            |  The activity the user was performing: bike sit, stand, walk, stairsup, stairsdown and null

# Structured Streaming

### Mock File Streaming (Throttle)

In [28]:
streaming = (
    spark
    .readStream
    .schema(dataSchema)
    .option('maxFilesPerTrigger', 1)
    .json('/resources/data/activity-data/')
)

### Simple Aggregations

In [29]:
# set partitions
spark.conf.set('spark.sql.shuffle.partitions', 5)

In [30]:
activityCounts = streaming.select('index').distinct()
activityQuery = (
    activityCounts.writeStream
    .queryName('activity_counts_4')
    .format('memory')
    .outputMode('append')
    .start()
)

# activityQuery.awaitTermination()

IllegalArgumentException: Cannot start query with name activity_counts_4 as a query with that name is already active in this SparkSession

In [31]:
# activityQuery.awaitTermination()
activityQuery.stop()

In [32]:
from time import sleep
for x in range(5):
    spark.sql("SELECT * FROM activity_counts_4").show()
    sleep(1)

+-----+
|index|
+-----+
|  115|
|  163|
|  436|
|  476|
|  555|
|  603|
|  643|
|  679|
|  719|
|  959|
| 1119|
| 1477|
| 1546|
| 1626|
| 1613|
| 1946|
| 2066|
| 2247|
| 2688|
| 2791|
+-----+
only showing top 20 rows

+-----+
|index|
+-----+
|  115|
|  163|
|  436|
|  476|
|  555|
|  603|
|  643|
|  679|
|  719|
|  959|
| 1119|
| 1477|
| 1546|
| 1626|
| 1613|
| 1946|
| 2066|
| 2247|
| 2688|
| 2791|
+-----+
only showing top 20 rows

+-----+
|index|
+-----+
|  115|
|  163|
|  436|
|  476|
|  555|
|  603|
|  643|
|  679|
|  719|
|  959|
| 1119|
| 1477|
| 1546|
| 1626|
| 1613|
| 1946|
| 2066|
| 2247|
| 2688|
| 2791|
+-----+
only showing top 20 rows

+-----+
|index|
+-----+
|  115|
|  163|
|  436|
|  476|
|  555|
|  603|
|  643|
|  679|
|  719|
|  959|
| 1119|
| 1477|
| 1546|
| 1626|
| 1613|
| 1946|
| 2066|
| 2247|
| 2688|
| 2791|
+-----+
only showing top 20 rows

+-----+
|index|
+-----+
|  115|
|  163|
|  436|
|  476|
|  555|
|  603|
|  643|
|  679|
|  719|
|  959|
| 1119|
| 1477|
| 1546|


### Complex Aggregation

In [33]:
from pyspark.sql.functions import expr

simpleTransform = (
    streaming
    .withColumn('stairs', expr("gt like '%stairs%'"))
    .where('stairs')
    .where('gt is not null')
    .select('gt', 'model', 'arrival_time', 'creation_time')
    .writeStream
    .queryName('simple_transform')
    .format('memory')
    .outputMode('append')
    .start()
)

In [34]:
for x in range(2):
    spark.sql("SELECT * FROM simple_transform").show()
    sleep(1)

+--------+------+-------------+-------------------+
|      gt| model| arrival_time|      creation_time|
+--------+------+-------------+-------------------+
|stairsup|nexus4|1424687983719|1424687981726802718|
|stairsup|nexus4|1424687984000|1424687982009853255|
|stairsup|nexus4|1424687984404|1424687982411977009|
|stairsup|nexus4|1424687984805|1424687982814351277|
|stairsup|nexus4|1424687985210|1424687983217500861|
|stairsup|nexus4|1424687985620|1424687983620332892|
|stairsup|nexus4|1424687986016|1424687984023164923|
|stairsup|nexus4|1424687986420|1424687984425874884|
|stairsup|nexus4|1424687986820|1424687984828822915|
|stairsup|nexus4|1424687987225|1424687985231654946|
|stairsup|nexus4|1424687987625|1424687985634469017|
|stairsup|nexus4|1424687987992|1424687986002114280|
|stairsup|nexus4|1424687988191|1424689834237427627|
|stairsup|nexus4|1424687988392|1424689834438660537|
|stairsup|nexus4|1424687988592|1424689834640076553|
|stairsup|nexus4|1424687988794|1424689834841675674|
|stairsup|ne

In [35]:
deviceModelStats = (
    streaming
    .groupBy('gt')
    .count()
    .writeStream
    .queryName('device_counts')
    .format('memory')
    .outputMode('complete')
    .start()
)

In [26]:
deviceModelStats.stop()

In [36]:
for x in range(10):
    spark.sql("SELECT * FROM device_counts").show()
    sleep(1)

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|36929|
|     stand|34154|
|stairsdown|28094|
|      walk|39768|
|  stairsup|31357|
|      null|31343|
|      bike|32390|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|49238|
|     stand|45539|
|stairsdown|37459|
|      walk|53024|
|  stairsup|41809|
|      null|41791|
|      bike|43187|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|73855|
|     stand|68309|
|stairsdown|56192|
|      walk|79536|
|  stairsup|62710|
|      null|62688|
|      bike|64781|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|86163|
|     stand|79694|
|stairsdown|65557|
|      walk|92792|
|  stairsup|73162|
|      null|73136|
|      bike|75579|
+----------+-----+

+----------+------+
|        gt| count|
+----------+------+
|       sit|110778|
|     stand|102464|
|stairsdown| 84286|
|      walk|119304|
|  stairsup| 94067|


In [14]:
spark.stop()